# Import libs

In [4]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import scanpy as sc
import anndata as ad
import tangram as tg

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import torch
torch.cuda.empty_cache()
device = torch.device("cpu")

In [6]:
# project_location = "e:/projects/stloc/"
project_location = "./stloc/"

# Load data

## Spatial data

In [48]:
df_st = pd.read_csv(project_location + "data/merfish/merfishSpatial.csv", sep=" ")
coords = df_st['coord']
df_st.drop(columns=['coord'], inplace=True)
df_st.head()

,Ace2,Adora2a,Aldh1l1,Amigo2,Ano3,Aqp4,Ar,Arhgap36,Avpr1a,Avpr2,...,ODImmature1,ODImmature2,Microglia,ODMature2,ODMature1,Endothelial3,ODMature3,ODMature4,Endothelial2,Ependymal
0,0.000000,0.631323,2.783859,4.017283,0.707772,2.754597,1.262664,14.922339,0.707772,0.0,...,0,0,0,1,1,0,0,0,0,0
1,0.000000,0.000000,5.027816,0.000000,0.000000,10.056050,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.000000,0.000000,0.000000,0.000000,3.856702,0.000000,0.854257,0.427118,0.0,...,0,0,0,1,0,0,0,0,0,0
3,0.000000,0.000000,1.465245,0.732607,0.000000,2.323503,0.795436,0.732607,0.732607,0.0,...,0,0,0,1,0,0,0,0,0,0
4,6.311896,7.574326,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,1,0,0,0,0


In [49]:
reference = pd.read_csv(project_location + "data/merfish/markerGene_for_merfish_data.csv")

reference.drop(columns=['p_value'], inplace=True)
# reference.drop(range(148,168))
reference.drop(reference.index[reference.cell_type=='EpendymalInhibitory'].tolist(), inplace=True)
reference['cell_type'] = reference['cell_type'].astype(str).str.replace(" ", "")
reference.head()

,cell_type,marker_gene
0,Astrocyte,Aldh1l1
1,Astrocyte,Aqp4
2,Astrocyte,Cxcl14
3,Astrocyte,Mlc1
4,Astrocyte,Ttyh2


In [50]:
genes = df_st.columns[:155]
markers = reference.groupby('cell_type').agg(list).marker_gene
celltypes = reference.cell_type.unique().tolist()
expressions = df_st.drop(columns=celltypes)
expressions = MinMaxScaler().fit_transform(expressions)
counts_st = pd.DataFrame(expressions, columns=genes)
cellcount = pd.DataFrame(np.sum(df_st[celltypes], axis=1), columns=['cellcount'])

In [86]:
# df_st.shape
zero = (cellcount==0).sum()
print('zero count is',zero)


zero count is cellcount    224
dtype: int64


## Single-cell data

In [52]:
df_sc = pd.read_csv(project_location + "data/merfish/merfishVisium.csv")

In [53]:
counts_sc = df_sc[genes]
obs_sc = df_sc[[x for x in df_sc.columns if x not in genes]]
obs_sc.head()

,Cell_ID,Animal_ID,Animal_sex,Behavior,Bregma,Centroid_X,Centroid_Y,Cell_class,Neuron_cluster_ID
0,1f9a8c19-b089-43d1-b609-7e791dc2c70f,1,Female,Naive,-0.24,-3749.176078,-3749.458442,Astrocyte,NaN
1,b13e98f4-5c2b-4e96-985e-3e93aedc7221,1,Female,Naive,-0.24,-3746.226210,-3742.259347,Inhibitory,I-1
2,d06cb29e-10ee-4bbc-b74e-90237999ef4b,1,Female,Naive,-0.24,-3742.897643,-3790.648737,Inhibitory,I-13
3,12e2a165-57c7-4f37-96dd-23f6574af4ba,1,Female,Naive,-0.24,-3738.423005,-3773.259265,Inhibitory,I-19
4,14a0f396-b13d-4d45-89a3-86c2047bf3f9,1,Female,Naive,-0.24,-3736.407620,-3895.590306,Pericytes,NaN


In [54]:
obs_sc.shape
obs_sc.loc[0,"Cell_class"]

'Astrocyte'

# Tangram

In [55]:
adata_sc = ad.AnnData(counts_sc)
adata_st = ad.AnnData(counts_st, obs=cellcount)
print(adata_sc.shape)
print(adata_st.shape)

(6412, 155)
(1249, 155)


In [82]:
adata_st.obs.head()
# zeroCount=(obs_sc["Cell_class"]==0).sum()


AnnData object with n_obs × n_vars = 1249 × 155
    obs: 'cellcount', 'uniform_density', 'rna_count_based_density'
    var: 'n_cells', 'sparsity'
    uns: 'training_genes', 'overlap_genes'

In [57]:
tg.pp_adatas(adata_sc, adata_st, genes=None)

INFO:root:155 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:155 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [58]:
adata_map_rna = tg.map_cells_to_space(adata_sc, adata_st, device=device, num_epochs=500)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 155 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.508, KL reg: 0.151
Score: 0.983, KL reg: 0.001
Score: 0.988, KL reg: 0.000
Score: 0.989, KL reg: 0.000
Score: 0.989, KL reg: 0.000


INFO:root:Saving results..


In [20]:
print(adata_map_rna.to_df().shape)
adata_map_rna.to_df().head()

(6412, 1249)


,0,1,2,3,4,5,6,7,8,9,...,1239,1240,1241,1242,1243,1244,1245,1246,1247,1248
0,8.328721e-08,2.345741e-07,3.763995e-07,1.534281e-07,3.448148e-07,7.609757e-08,2.314369e-06,9.884618e-07,2.987654e-07,3.635387e-07,...,3.126729e-07,6.026074e-08,5.585241e-07,8.979840e-07,2.543524e-07,1.001547e-06,9.245763e-07,9.813517e-01,4.246305e-07,1.905592e-07
1,1.600638e-07,1.611434e-07,5.432863e-07,5.878340e-08,9.742198e-07,1.037845e-07,3.441358e-08,3.987220e-07,4.245252e-07,3.033094e-07,...,2.034778e-07,3.235256e-07,1.191153e-05,4.659870e-07,4.481985e-07,4.294650e-07,5.965369e-07,9.947686e-01,1.547774e-03,7.502450e-07
2,1.151779e-07,1.603974e-07,3.144310e-08,1.395429e-07,6.045381e-08,5.261430e-07,1.100216e-07,3.923840e-07,1.010913e-07,8.473393e-07,...,4.430716e-07,1.626301e-08,3.669247e-07,2.575632e-07,5.127390e-08,7.476120e-08,9.025911e-07,1.329402e-07,9.986860e-01,2.314991e-07
3,8.212434e-08,1.304487e-07,1.189635e-07,9.477837e-08,8.108015e-08,1.768097e-06,5.588211e-07,1.819106e-06,2.220222e-07,8.417074e-07,...,8.079193e-08,8.080426e-08,9.802468e-08,1.009974e-05,1.547398e-07,7.011547e-07,4.399845e-07,5.459723e-07,9.970515e-01,1.681645e-07
4,5.007190e-06,9.737385e-07,9.019274e-07,4.612345e-07,5.595609e-07,2.141731e-06,2.585484e-06,5.223499e-04,3.639371e-06,1.441093e-06,...,2.324459e-06,1.997343e-06,1.431437e-06,8.160324e-06,7.159709e-06,6.004784e-06,2.237070e-06,5.241531e-07,2.664614e-06,1.659714e-05


In [59]:
adata_map_cellcount = tg.map_cells_to_space(
    adata_sc,
    adata_st,
    target_count=adata_st.obs.cellcount.sum(),
    density_prior=np.array(adata_st.obs.cellcount) / adata_st.obs.cellcount.sum(),
    device=device,
    num_epochs=500
)
probabilityDf= adata_map_cellcount.to_df()

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 155 genes and customized density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.508, KL reg: 0.321
Score: 0.970, KL reg: 0.003
Score: 0.975, KL reg: 0.003
Score: 0.975, KL reg: 0.003
Score: 0.976, KL reg: 0.003


INFO:root:Saving results..


In [60]:
probabilityDf.shape

(6412, 1249)

# probability analysis

In [72]:
np.argmax(probabilityDf.iloc[2])
probabilityDf.iloc[2,1247]

Nrows= len(df_st)
Ncols= len(celltypes)

deconvolveDf = pd.DataFrame(np.zeros((Nrows,Ncols)),columns=celltypes)
for i in range(len(probabilityDf)):
    spotNo = np.argmax(probabilityDf.iloc[i])
    cellClass= obs_sc.loc[i,"Cell_class"]
    deconvolveDf.loc[spotNo,cellClass] = deconvolveDf.loc[spotNo,cellClass] +1


deconvolveDf.shape

(1249, 16)

In [73]:
deconvolveDf.head()

,Astrocyte,Inhibitory,Pericytes,Ambiguous,Endothelial1,Excitatory,ODImmature1,ODImmature2,Microglia,ODMature2,ODMature1,Endothelial3,ODMature3,ODMature4,Endothelial2,Ependymal
0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [74]:
for i in range(len(deconvolveDf)):
    deconvolveDf.iloc[i] = deconvolveDf.iloc[i]/deconvolveDf.iloc[i].sum()



In [77]:
deconvolveDf.head()
deconvolveDf.to_csv("tangramDeconvolved.csv",index= False)
deconvolveDf.isnull().sum(axis = 0)


Astrocyte       205
Inhibitory      205
Pericytes       205
Ambiguous       205
Endothelial1    205
Excitatory      205
ODImmature1     205
ODImmature2     205
Microglia       205
ODMature2       205
ODMature1       205
Endothelial3    205
ODMature3       205
ODMature4       205
Endothelial2    205
Ependymal       205
dtype: int64